<a href="https://colab.research.google.com/github/jim-krehl/stylegan3-fork/blob/main/StyleGAN3-fork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://colab.research.google.com/github/dvschultz/stylegan3/blob/main/SG3.ipynb#scrollTo=1tgIuCLRehuP

In [ ]:
!nvidia-smi -L

GPU 0: NVIDIA A100-SXM4-40GB (UUID: GPU-e763e48c-6ee0-87c5-e956-e3e30cdf60c0)


In [ ]:
from google.colab import drive
drive.mount ('/content/drive')

In [ ]:
BASE_DIR = '/content/drive/MyDrive/projects/HEIMR/colab-StyleGAN3/'
STYLEGAN3_DIR = BASE_DIR + 'stylegan3/'
PRETRAINING_DIR = STYLEGAN3_DIR + 'pretrained/'

SOURCE_IMAGE_DIR = BASE_DIR + '../source-images.1'

In [ ]:
!pip uninstall jax jaxlib -y
!pip install "jax[cudall_cudnn805]==0.3.10" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip uninstall torch torchvision -y
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html
!pip install timm==0.4.12 ftfy==6.1.1 ninja==1.10.2 imageio-ffmpeg opensimplex setuptools==59.5.0

In [ ]:
import os

if not os.path.isdir (STYLEGAN3_DIR):
  %cd $BASE_DIR
  !git clone https://github.com/dvschultz/stylegan3
else:
  print ("stylegan3 already cloned")

if not os.path.isfile (PRETRAINING_DIR + 'wikiart.pkl'):
  if not os.path.isdir (PRETRAINING_DIR):
    !mkdir $PRETRAINING_DIR

  %cd $PRETRAINING_DIR
  !gdown "1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU"
else:
  print ("wikiart.pkl already downloaded") 

In [ ]:
%cd $STYLEGAN3_DIR

if not os.path.isdir ('./datasets/'):
  !mkdir datasets

!python dataset_tool.py --source=$SOURCE_IMAGE_DIR --dest=./datasets/source-images-1.zip --resolution=2048x2048

In [ ]:
%cd $STYLEGAN3_DIR

if not os.path.isdir ('./results/'):
  !mkdir results-1

dataset='./datasets/source-images-1-256x256'
resume_pkl='./snapshots/snapshot-00006-256x256-000176.pkl'
!python train.py --outdir=./results/ --cfg=stylegan3-t --data=$dataset --gpus=1 --batch=4 --batch-gpu=4 --gamma=10.0 --mirror=1 --kimg=5000 --snap=1 --resume=$resume_pkl

In [ ]:
%cd $STYLEGAN3_DIR

gen_img_dir='./generated-images/batch.0/'
network='./snapshots/snapshot-00007-256x256-000208.pkl'
!python gen_images.py --outdir=$gen_img_dir --network=$network --seeds=0-1000